In [406]:
import os
from pathlib import Path
os.chdir(str(Path(os.getcwd()).parent.absolute()))
from datetime import date,datetime,timedelta
from importlib import reload
from dateutil.relativedelta import relativedelta

import cudf
import dask_cudf
import pandas as pd
import cupy
import cuml
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error
import numpy as np

from src.metric.loss import symmetric_mean_absolute_percentage_error as smape
from multiprocessing import Manager,Queue,Pool

In [2]:
from tsfresh.feature_extraction import extract_features

In [3]:
df=pd.read_csv('data/train.csv')

In [4]:
df

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [5]:
df['first_day_of_month']=pd.to_datetime(df['first_day_of_month'])

In [6]:
features=['row_id','cfips','first_day_of_month','mcb','active']

In [7]:
df_=(
    df.rename(mapper={'microbusiness_density':'mcb'},axis=1)
    .sort_values('row_id')
    .assign(next_mcb=lambda x:x.groupby('cfips')['mcb'].shift(-1))
    .assign(target=lambda x:(x.next_mcb-x.mcb)/x.mcb)
)

In [8]:

features=['row_id', 'cfips', 'first_day_of_month', 'target','active']

In [9]:
df_label=df_[['row_id','target','first_day_of_month']].dropna()
df_label['target']=df_label['target'].clip(-.05,.05)

In [10]:
df_feature_1=df_[features].copy(deep=True).rename({'first_day_of_month':'dt1','target':'target_1'},axis=1)
df_feature_2=df_[features].copy(deep=True).rename({'first_day_of_month':'dt2','target':'target_2'},axis=1).drop(['active'],axis=1)

In [11]:
df_join=pd.merge(
    df_feature_1.drop(['row_id'],axis=1),
    df_feature_2,
    on='cfips'
).loc[lambda x:x.dt2>x.dt1].sort_values(['row_id','dt1'])[['row_id','target_1','dt1','active']].fillna(0)

In [12]:
df_join['target_1']=df_join['target_1'].clip(-.05,.05)

In [13]:
df_ts_feature=extract_features(df_join.drop(['active'],axis=1),column_id='row_id',column_sort='dt1')

Feature Extraction: 100%|██████████| 160/160 [02:47<00:00,  1.05s/it]


In [14]:
df_ft_active=df_join.groupby('row_id').agg(
    # last_active=('active',),
    # first_active=('active',pd.),
    mean_active=('active',np.mean),
    med_active=('active',np.median),
    max_active=('active',np.max),
    min_active=('active',np.min),
)
df_ft_active['last_active']=df_join.groupby('row_id')['active'].last()
df_ft_active['first_active']=df_join.groupby('row_id')['active'].first()
df_ft_active.reset_index(inplace=True)

In [15]:
df_ft_active

,row_id,mean_active,med_active,max_active,min_active,last_active,first_active
0,10001_2019-09-01,6624.000000,6624.0,6624,6624,6624,6624
1,10001_2019-10-01,6648.000000,6648.0,6672,6624,6672,6624
2,10001_2019-11-01,6685.333333,6672.0,6760,6624,6760,6624
3,10001_2019-12-01,6712.750000,6716.0,6795,6624,6795,6624
4,10001_2020-01-01,6719.000000,6744.0,6795,6624,6744,6624
...,...,...,...,...,...,...,...
119125,9015_2022-06-01,3496.882353,3461.5,3792,3266,3296,3440
119126,9015_2022-07-01,3491.085714,3440.0,3792,3266,3294,3440
119127,9015_2022-08-01,3486.972222,3414.0,3792,3266,3343,3440
119128,9015_2022-09-01,3482.540541,3388.0,3792,3266,3323,3440


In [16]:
df_final_feature=pd.merge(
    pd.merge(
        df_ft_active,
        df_ts_feature.reset_index(names='row_id'),
        on='row_id'
    ),
    df_label,
    on='row_id'
)

## Training

In [17]:
pivot_train=datetime(2022,1,1)

In [260]:
df_train=df_final_feature.loc[lambda x:x.first_day_of_month<pivot_train].sample(frac=1).reset_index(drop=True)

In [261]:
df_eval=df_final_feature.loc[lambda x:x.first_day_of_month>pivot_train].sample(frac=1).reset_index(drop=True)


In [396]:
xgb=XGBRegressor(tree_method='gpu_hist', gpu_id=0,predictor='gpu_predictor',        objective='reg:pseudohubererror',
        #objective='reg:squarederror',
        n_estimators=7000,
        learning_rate=0.5,
        max_leaves = 14,
        max_depth=18,
        # subsample=0.50,
        colsample_bytree=0.50,
        max_bin=512,
        reg_lambda=1.5,
        gamma=1.5,
        n_jobs=2,
)

In [397]:
xgb.fit(
    df_train.iloc[:int(0.7*len(df_train))].drop(['first_day_of_month','target','row_id'],axis=1),
    df_train.iloc[:int(0.7*(len(df_train)))]['target']
)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=1.5, gpu_id=0,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.5, max_bin=512,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=18, max_leaves=14, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=7000, n_jobs=2,
             num_parallel_tree=1, objective='reg:pseudohubererror',
             predictor='gpu_predictor', ...)

In [355]:
val_df=df_train[xgb.feature_names_in_]
max_val= np.nanmax(val_df[val_df!=np.inf])
min_val=np.nanmin(val_df[val_df!=-np.inf])

In [358]:
df_train.iloc[int(0.7*(len(df_train))):].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:max_val,-np.inf:min_val}).min().min()

-5747752743894727.0

In [359]:
mean_squared_error(
    df_train.iloc[int(0.7*(len(df_train))):]['target'],
    xgb.predict(df_train.iloc[int(0.7*(len(df_train))):].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:max_val,-np.inf:min_val}))
)

0.0004094087607162363

In [360]:
xgb.predict(df_train.iloc[int(0.7*(len(df_train))):].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:0,-np.inf:0}))


array([0.00061256, 0.00061256, 0.00061256, ..., 0.00061256, 0.00061256,
       0.00061256], dtype=float32)

In [361]:
mean_squared_error(
    df_eval.iloc[:]['target'],
    xgb.predict(df_eval.iloc[:].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:0,-np.inf:0}))
)

0.00029861688335251383

In [316]:
df_eval_in_time=df_eval[['row_id','first_day_of_month','target']]
df_eval_in_time['pred']=xgb.predict(df_eval.iloc[:].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:0,-np.inf:0})).tolist()


/tmp/ipykernel_338092/3217451328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eval_in_time['pred']=xgb.predict(df_eval.iloc[:].drop(['first_day_of_month','target','row_id'],axis=1).replace({np.inf:0,-np.inf:0})).tolist()


In [317]:
df_importance=pd.DataFrame(
    {
        'feature':xgb.feature_names_in_,
        'score':xgb.feature_importances_
    }
).sort_values('score',ascending=False)

In [344]:
df_train[xgb.feature_names_in_].max()

mean_active                                                    1.139015e+06
med_active                                                     1.138541e+06
max_active                                                     1.167744e+06
min_active                                                     1.119817e+06
last_active                                                    1.167744e+06
                                                                   ...     
target_1__permutation_entropy__dimension_5__tau_1              3.178054e+00
target_1__permutation_entropy__dimension_6__tau_1              3.135494e+00
target_1__permutation_entropy__dimension_7__tau_1              3.091042e+00
target_1__query_similarity_count__query_None__threshold_0.0             NaN
target_1__mean_n_absolute_max__number_of_maxima_7              5.000000e-02
Length: 789, dtype: float64

In [318]:
df_eval_oot=pd.merge(
    df_label,
    df.loc[lambda x:x.first_day_of_month>=pivot_train][['row_id', 'microbusiness_density','cfips']],
    on='row_id'
)

In [398]:
df_continuos=pd.merge(
    df_label,
    df.loc[lambda x:x.first_day_of_month<pivot_train][['row_id', 'microbusiness_density','cfips']],
    on='row_id'
)

In [399]:
from src import train

In [400]:
df_continuos

,row_id,target,first_day_of_month,microbusiness_density,cfips
0,10001_2019-08-01,0.007246,2019-08-01,4.995701,10001
1,10001_2019-09-01,0.013189,2019-09-01,5.031902,10001
2,10001_2019-10-01,0.005178,2019-10-01,5.098270,10001
3,10001_2019-11-01,-0.007505,2019-11-01,5.124666,10001
4,10001_2019-12-01,-0.013513,2019-12-01,5.086203,10001
...,...,...,...,...,...
90896,9015_2021-08-01,-0.003922,2021-08-01,3.555913,9015
90897,9015_2021-09-01,-0.000909,2021-09-01,3.541968,9015
90898,9015_2021-10-01,0.006062,2021-10-01,3.538750,9015
90899,9015_2021-11-01,-0.008436,2021-11-01,3.560204,9015


In [401]:
reload(train)

<module 'src.train' from '/var/DSTeam/locnt2/personal/micro_business/src/train.py'>

In [402]:
for dt in [pivot_train+relativedelta(months=i) for i in range(0,10)]:
    df_this_dt=train.create_feature_ts_for_dt(
        df_continuos=df_continuos,
        dt=dt,
        df_feature_active=df_ft_active,
        target_col='target',
        mcb_col='microbusiness_density',
        time_col='first_day_of_month'
    )
    pred_target=xgb.predict(
        df_this_dt[xgb.feature_names_in_].replace({np.inf:max_val,-np.inf:min_val})
    )
    df_next=pd.DataFrame(
        {
            'target':pred_target.tolist(),
            'first_day_of_month':dt,
            'cfips':df_this_dt['cfips'],
        }
    ).assign(row_id=lambda x:x['cfips'].apply(str)+'_'+x['first_day_of_month'].dt.strftime('%Y-%m-%d'))
    
    df_next_with_prev_mcb=pd.merge(
        df_next,
        df_continuos.groupby('cfips')['microbusiness_density'].last().reset_index().rename({'microbusiness_density':'prev_mcb'},axis=1),
        on='cfips'
    ).assign(microbusiness_density=lambda x:(x['target']+1)*x['prev_mcb'])[['row_id','target','first_day_of_month','microbusiness_density','cfips']]
    
    df_continuos=pd.concat([df_continuos,df_next_with_prev_mcb])


Feature Extraction: 100%|██████████| 157/157 [00:06<00:00, 25.21it/s]


In [403]:
df_eval_with_pred=pd.merge(
    df_eval_oot[['row_id','cfips','first_day_of_month','microbusiness_density']].rename({'microbusiness_density':'truth'},axis=1),
    df_continuos[['row_id','cfips','first_day_of_month','microbusiness_density']].rename({'microbusiness_density':'pred'},axis=1),
    on=['row_id','cfips','first_day_of_month']
)

In [404]:
x=50
df_eval_with_pred[x:x+10]

,row_id,cfips,first_day_of_month,truth,pred
50,1005_2022-06-01,1005,2022-06-01,1.191678,1.156130
51,1005_2022-07-01,1005,2022-07-01,1.216926,1.156838
52,1005_2022-08-01,1005,2022-08-01,1.196728,1.157547
53,1005_2022-09-01,1005,2022-09-01,1.206827,1.158256
54,1007_2022-01-01,1007,2022-01-01,1.214165,1.210012
55,1007_2022-02-01,1007,2022-02-01,1.236650,1.210753
56,1007_2022-03-01,1007,2022-03-01,1.264755,1.211495
57,1007_2022-04-01,1007,2022-04-01,1.253513,1.212237
58,1007_2022-05-01,1007,2022-05-01,1.247892,1.212979
59,1007_2022-06-01,1007,2022-06-01,1.275998,1.213722


In [405]:
smape(df_eval_with_pred.truth,df_eval_with_pred.pred)

0.037222943631900415

In [407]:
manager=Manager()
dict_smape=manager.dict()

In [408]:
def get_sampe_for_cfips(cfips:int):
    tmp_df=df_eval_with_pred.loc[lambda x:x.cfips==cfips]
    dict_smape[cfips]=smape(tmp_df.truth,tmp_df.pred)

In [409]:
list_cfips=list(set(df_eval_with_pred.cfips))

In [411]:
with Pool(processes=32) as p:
    p.map(get_sampe_for_cfips,list_cfips)

In [414]:
normal_dict_smape={}
for key in dict_smape.keys():
    normal_dict_smape[key]=dict_smape[key]

In [516]:
list_modified_cfips=pd.DataFrame({
    'cfips':[x for x,_ in sorted(normal_dict_smape.items(),key=lambda x:x[1])[-100:]]
    }
)

In [517]:
list_modified_cfips

,cfips
0,22123
1,20033
2,19175
3,2230
4,30085
...,...
95,51043
96,32510
97,13239
98,38087


In [518]:
df_last_val=pd.merge(
    df_label,
    df.loc[lambda x:x.first_day_of_month<pivot_train][['row_id', 'microbusiness_density','cfips']],
    on='row_id'
).groupby('cfips')['microbusiness_density'].last().reset_index()

In [519]:
df_modified_cfips=pd.merge(
    df_last_val,
    list_modified_cfips,
    on='cfips'
)

In [520]:
df_=pd.merge(
    df_eval_with_pred,
    df_modified_cfips,
    on='cfips',
    how='left'
)
df_['microbusiness_density'].fillna(value=df_.pred,inplace=True)

In [521]:
df_

,row_id,cfips,first_day_of_month,truth,pred,microbusiness_density
0,10001_2022-01-01,10001,2022-01-01,6.218161,6.260361,6.260361
1,10001_2022-02-01,10001,2022-02-01,6.352220,6.264196,6.264196
2,10001_2022-03-01,10001,2022-03-01,6.445699,6.268033,6.268033
3,10001_2022-04-01,10001,2022-04-01,6.502221,6.271873,6.271873
4,10001_2022-05-01,10001,2022-05-01,6.481931,6.275715,6.275715
...,...,...,...,...,...,...
28201,9015_2022-05-01,9015,2022-05-01,3.524116,3.540994,3.540994
28202,9015_2022-06-01,9015,2022-06-01,3.521978,3.543163,3.543163
28203,9015_2022-07-01,9015,2022-07-01,3.574369,3.545334,3.545334
28204,9015_2022-08-01,9015,2022-08-01,3.552985,3.547506,3.547506


In [522]:
smape(df_.truth,df_.microbusiness_density)

0.03725350462173412